# Déploiement AKS

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

Le service complètement managé **Azure Kubernetes Service (AKS)** facilite le déploiement et la gestion d'applications conteneurisées. <br>
Il offre une **expérience d'intégration continue et de livraison continue (CI/CD) Kubernetes serverless**, ainsi qu'une **sécurité et une gouvernance de classe Entreprise**. <br>
Réunissez vos équipes dédiées aux déploiements et aux opérations sur une même plateforme pour rapidement créer, livrer et mettre à l'échelle des applications en toute confiance.

Azure Kubernetes Service est idéal pour les déploiements de production à grande échelle. Utilisez Azure Kubernetes Service si vous avez besoin d’une ou de plusieurs des fonctionnalités suivantes :
- Temps de réponse rapide.
- Mise à l’échelle automatique du service déployé.
- Options d’accélération matérielle, telles que le GPU et les FPGA (Field-Programmable Gate Array).

Documentation : https://azure.microsoft.com/fr-fr/services/kubernetes-service/<br>
Checklist AKS :
https://www.the-aks-checklist.com/<br>
<br>
> Informations sur le déploiement des modèles en AKS : https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-deploy-azure-kubernetes-service

### Architecture

<img src="https://docs.microsoft.com/fr-fr/azure/architecture/reference-architectures/ai/_images/python-model-architecture.png">

## 1. Infos

In [2]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import AksWebservice
import azureml.core
import json

print('Version Azure ML :', azureml.core.VERSION)

Version Azure ML : 1.2.0


In [3]:
import datetime
now = datetime.datetime.now()
print(now)

2020-04-17 07:59:02.020814


## 2. Workspace


In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

workshopAML2020
workshopAML2020-rg
westeurope
70b8f39e-8863-49f7-b6ba-34a80799550c


## 3. Enregistrement du modèle de ML

In [10]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # Nom du fichier local
                       model_name = "sklearn_regression_model.pkl", # Nom du fichier référencé dans Azure ML
                       tags = {'area': "diabetes", 'type': "regression", 'format': "Scikit-Learn pkl"},
                       description = "Modèle DIABETE régression Ridge",
                       workspace = ws)
print()
print("Nom du modèle :", model.name)
print("Description :", model.description)
print("Version du modèle :", model.version)

Registering model sklearn_regression_model.pkl

Nom du modèle : sklearn_regression_model.pkl
Description : Modèle DIABETE régression Ridge
Version du modèle : 13


## 4. Scoring File

In [38]:
%%writefile score.py
import os
import pickle
import json
import numpy 
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
import time

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType

def init():
    global model
    #Print statement for appinsights custom traces:
    print ("model initialized" + time.strftime("%H:%M:%S"))
    
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)
  

# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = numpy.array(data)
        result = model.predict(data)
        print ("Prediction created" + time.strftime("%H:%M:%S"))
        # you can return any datatype as long as it is JSON-serializable
        return result.tolist()
    except Exception as e:
        error = str(e)
        print (error + time.strftime("%H:%M:%S"))
        return error

Overwriting score.py


## 5. Création du fichier d'environnement YAML

Lister les prérequis avec éventuellement les versions nécessaires.

In [50]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn==0.20.0'],
                                 pip_packages=['azureml-defaults>=1.0.45', 'inference-schema[numpy-support]']
                                )

In [51]:
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## 6. Création de la configuration d'inférence

In [52]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

## 7. Création et déploiement Azure Kubernetes Service (AKS)

La création ou l’attachement d’un cluster AKS est un processus à effectuer une seule fois pour votre espace de travail. Vous pouvez le réutiliser pour vos autres déploiements. Si vous supprimez le cluster ou le groupe de ressources dans lequel il se trouve, vous devrez recréer un cluster lors du prochain déploiement. Vous pouvez avoir plusieurs clusters AKS attachés à votre espace de travail.

In [22]:
# Configuration par défaut
prov_config = AksCompute.provisioning_configuration()

aks_name = 'instance-aks' #Nom de l'instance AKS à crééer

# Création du cluster AKS
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [23]:
%%time
aks_target.wait_for_completion(show_output = True)

Creating...........................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
CPU times: user 581 ms, sys: 56.6 ms, total: 638 ms
Wall time: 11min 49s


> Note : Prévoir 10 minutes de temps de traitement

In [24]:
print('Statut AKS :')
print()
print("Etat =", aks_target.provisioning_state)
print("Erreur =", aks_target.provisioning_errors)

Statut AKS :

Etat = Succeeded
Erreur = None


If you already have a cluster you can attach the service to it:

```python 
%%time
resource_id = '/subscriptions/<subscriptionid>/resourcegroups/<resourcegroupname>/providers/Microsoft.ContainerService/managedClusters/<aksservername>'
create_name= 'myaks4'
attach_config = AksCompute.attach_configuration(resource_id=resource_id)
aks_target = ComputeTarget.attach(workspace = ws, 
                                  name = create_name, 
                                  attach_configuration=attach_config)
## Wait for the operation to complete
aks_target.wait_for_provisioning(True)```

### Activation App Insights pour la télémétrie

In [25]:
aks_deployment_config = AksWebservice.deploy_configuration(enable_app_insights=True)

### Déploiement du modèle dans l'instance AKS

In [53]:
%%time
if aks_target.provisioning_state== "Succeeded": 
    aks_service_name ='modele-exemple12-aks'  # Nom unique du modèle déployé
    aks_service = Model.deploy(ws,
                               aks_service_name, 
                               [model], 
                               inference_config, 
                               aks_deployment_config, 
                               deployment_target = aks_target)    
    aks_service.wait_for_deployment(show_output = True)
    print(aks_service.state)
else:
    raise ValueError("AKS provisioning failed. Error: ", aks_service.error)

Running.........................................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 373 ms, sys: 60.7 ms, total: 433 ms
Wall time: 6min 36s


> Prévoir 4 minutes de temps de traitements

### Informations AKS :

In [54]:
print("Informations du modèle déployé en AKS :")
print(" - Endpoint :", aks_service.scoring_uri)
print(" - Statut :", aks_service.state)
print(" - Swagger :", aks_service.swagger_uri)

Informations du modèle déployé en AKS :
 - Endpoint : http://52.233.132.186:80/api/v1/service/modele-exemple12-aks/score
 - Statut : Healthy
 - Swagger : http://52.233.132.186:80/api/v1/service/modele-exemple12-aks/swagger.json


In [55]:
print("Liste des modèles déployés avec Azure ML :")
print()
for webservice_name in ws.webservices:
    print(webservice_name)

Liste des modèles déployés avec Azure ML :

modele-exemple12-aks
bostonhouseprice-aci


In [56]:
print("Logs AKS :")
print()
print(aks_service.get_logs())

Logs AKS :

/usr/sbin/nginx: /azureml-envs/azureml_dfa8f307167610c047619b5131f08dcb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfa8f307167610c047619b5131f08dcb/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfa8f307167610c047619b5131f08dcb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfa8f307167610c047619b5131f08dcb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_dfa8f307167610c047619b5131f08dcb/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-04-17T08:46:57,953924223+00:00 - gunicorn/run 
2020-04-17T08:46:57,955948810+00:00 - nginx/run 
2020-04-17T08:46:57,957476275+00:00 - rsyslog/run 
2020-04-17T08:46:57,967645311+00:00 - iot-server/r

## 8. Réalisation de plusieurs tests d'appel au modèle de ML exposé dans AKS

In [57]:
print("Scoring endpoint AKS :", aks_service.scoring_uri)

Scoring endpoint AKS : http://52.233.132.186:80/api/v1/service/modele-exemple12-aks/score


> Remarque : Les clefs sont visibles dans la section Endpoint Azure ML Studio.

### Test 1

In [67]:
test_sample1 = json.dumps({'data': [
    [1,18,13,45,54,6,57,8,8,50], 
    [10,9,8,37,6,15,4,5,2,43]
]})
test_sample1 = bytes(test_sample1,encoding='utf8')

if aks_service.state == "Healthy":
    prediction1 = aks_service.run(input_data=test_sample1)
    print(">> Prédictions du modèle via AKS :", prediction1)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

>> Prédictions du modèle via AKS : [9276.53327694989, 14169.060737772432]


### Test 2

In [68]:
test_sample2 = json.dumps({'data': [
    [2,28,13,45,54,6,57,8,8,50], 
    [10,9,8,37,6,45,4,3,2,43]
]})
test_sample2 = bytes(test_sample2,encoding='utf8')

if aks_service.state == "Healthy":
    prediction2 = aks_service.run(input_data=test_sample2)
    print(">> Prédictions du modèle via AKS :", prediction2)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

>> Prédictions du modèle via AKS : [7909.812215422598, 11478.00168731773]


### Test 3

In [69]:
test_sample3 = json.dumps({'data': [
    [24,48,3,45,54,6,57,8,8,50], 
    [13,5,55,37,6,45,4,3,2,43]
]})
test_sample3 = bytes(test_sample3,encoding='utf8')

if aks_service.state == "Healthy":
    prediction3 = aks_service.run(input_data=test_sample3)
    print(">> Prédictions du modèle via AKS :", prediction3)
else:
    raise ValueError("Service deployment isn't healthy, can't call the service. Error: ", aks_service.error)

>> Prédictions du modèle via AKS : [1107.1856643704439, 31507.05936899806]


## 9. Accés aux données de télémétries AppInsights
Aller dans le portail Azure pour visualiser les informations de télémétries disponibles depuis AKS

<img src="https://github.com/retkowsky/images/blob/master/aksresults.jpg?raw=true">

## 10. Désactivation

In [23]:
#aks_service.delete()
#aci_service.delete()
#model.delete()
#endpoint.delete()

<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">